# Grammar and Spelling

- count the number of corrected words per user
- normalized by the total count of words by the user

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../../giicg.db')
prompts = pd.read_sql("SELECT ep.*, u.age, u.work_exp_years FROM translated_prompts ep JOIN users u ON ep.user_id = u.user_id", conn)
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language,age,work_exp_years
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en,19-25,3.0
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en,19-25,3.0
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en,19-25,3.0
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en,19-25,3.0
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en,19-25,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
750,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en,36-40,3.0
751,1845,37,user,\n nun möchte ich judgement balancing m...,Now I want to bring judgement balancing into t...,,,Woman (cisgender),29,de,26-30,None
752,1847,37,user,\n ich sehe keine veränderung im Plot. Was ...,I don't see any change in the plot.,,,Woman (cisgender),29,de,26-30,None
753,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en,19-25,1.0


In [2]:
user_prompts = (
    prompts
    #.fillna({'work_exp_years': 'None'})
    .groupby(['user_id', 'gender'])['conversational']
    .apply(' '.join)
    .reset_index()    # Reset index to create a DataFrame
)
user_prompts['n_words'] = user_prompts['conversational'].apply(lambda x: len(x.split()))
user_prompts = user_prompts[(user_prompts['gender'].isin(['Man (cisgender)', 'Woman (cisgender)']))]

user_prompts

,user_id,gender,conversational,n_words
0,6,Man (cisgender),"parsing data from python iterator, how it coul...",196
1,8,Man (cisgender),\n I am working on the problem of reconstru...,36
2,11,Woman (cisgender),Can you adapt the following code so that inste...,193
3,15,Man (cisgender),SET_ALL_TABLES action is currently not fetchin...,36
4,16,Woman (cisgender),I want to use Dummy Hot encoding to replace th...,542
5,25,Man (cisgender),whats the best way to encode and compress a ja...,107
6,28,Woman (cisgender),I have a pandas dataframe like this:\n\nI want...,561
7,29,Woman (cisgender),Now I want to bring judgement balancing into t...,61
9,31,Man (cisgender),How can I make use of an ObservableHQDatabaseC...,92
10,34,Man (cisgender),Blender and Python. I have a collection of hun...,1299


In [3]:
corrections = {
    'orthogogal': 'orthogonal',
    'Anothers' : 'Another',
    'follwoing' : 'following',
    'insted' : 'instead',
    'addtion' : 'addition',
    'develope' : 'develop',
    'familys' : 'families',
    'responsable' : 'responsible',
    'optimate' : 'optimize',
    'esier' : 'easier',
    'sentencs': 'sentences',
    'preveent' : 'prevent',
    'indexs': 'indices',
    'hundrets': 'hundreds',
    'orthotogonal' : 'orthogonal',
    'palatte' : 'plaette',
    'favoriting' : 'favoring',
    'orignal' : 'original',
    'resemblens' : 'resemblance',
    'appoinment' : 'appointment',
    'ealier' : 'earlier',
    'Rectlange' : 'Rectangle',
    'gesorächslauf' : 'chat history',
    'evry': 'every',
    'inlcude': 'include',
    'wrtie' : 'write',
    'tring' : 'trying',
    'exaple' : 'example',
    'modalitieees': 'modalities',
    'doenst' : 'doesn\'t',
    'actualy': 'actually',
    'paremeter': 'parameter',
    'experiement': 'experiment',
    'concatination' : 'concatenation',
    'impliment' : 'implement',
    'impilmentation' : 'implementation',
    'nessacary' : 'necessary',
    'caclulated' : 'calculated',
    'matrixes' : 'matrices',
    'writie' : 'write',
    'follws' : 'follows',
    'inlcuded' : 'included',
    'origional' : 'original',
    'thta' : 'that',
    'laike' : 'like',
    'wehre' : 'where',
    'reuslts':'results',
    'intersetd' : 'interested',
    'concatition' : 'concatenation',
    'paramters' : 'parameters',
    'implyin' : 'implying',
    'guassion' : 'gaussian',
    'thigns' : 'things',
    'dockerfule' : 'dockerfile',
    'snipped'  : 'snippet',
    'viseos' : 'videos',
    'phne' : 'without',
    'deday' : 'decay',
    'seperate' : 'separate',
    'apporach' : 'approach',
    'indiviually' : 'individually',
    'allegeded' : 'alleged',
    'testset' : 'test set',
    'classificaiton' : 'classification',
    'optimice' : 'optimize',
    'adaptbefore' : 'adapt before',
    'onlz' : 'only',
    'sepearte' : 'separate',
    'informaiton' : 'information',
    'validaiton' : 'validation',
    'overfitiing' : 'overfitting',
    'eahc' : 'each',
    'directorys' : 'directories',
    'direcrtorys' : 'directories',
    'manipultaet' : 'manipulated',
    'aere' : 'are',
    'withoud' : 'without',
    'thorugh' : 'thorugh',
    'please provide example hot to use' : 'please provide example how to use',
    'rewertie' : 'rewrite',
    'inforamtion' : 'information',
    'visualiysing' : 'visualizing',
    'achive' : 'achieve',
    'sohuellete' : 'silhouette',
    'autimatically' : 'automatically',
    'entrys' : 'entries',
    'visualisaitons' : 'visualizations',
    'seachr' : 'search',
    'searhc' : 'search',
    'shoulndt' : 'shouldn\'t',
    'llookng' : 'looking',
    'clutering' : 'clustering',
    'restuks' : 'results',
    'mehtod' : 'method',
    'delimimetr' : 'delimiter',
    'strucuture' : 'structure',
    'neccessary' : 'necessary',
    'extractzion' : 'extraction',
    'suddently' : 'suddenly',
    'miminal' : 'minimal',
    'anomalious' : 'anomalous',
    'staistical' : 'statistical',
    'vlaues' : 'values',
    'calsses' : 'classes',
    'ommitted' : 'omitted',
    'approcheas' : 'approaches',
    'inthe' : 'in the',
    'sceintificly' : 'scientifically',
    'soonn' : 'soon',
    'dissappear' : 'disappear',
    'tryping' : 'trying',
    'undefinded' : 'undefined',
    'lenght' : 'length',
    'overritten' : 'overwritten',
    'exisiting' : 'existing',
    'caluclate' : 'calculate',
    'optimise' : 'optimize',
    'HEader' : 'Header',
    'nich tunterstrichen' : 'no underscore',
    'carussel': 'carousel',
    'adress' : 'address',
    'blancs' : 'blanks',
    'formated' : 'formatted',
    'unmanipluated': 'unmanipulated',
    'additonal': 'additional',

}

In [4]:
import re

# Pre-compile regex for each misspelling for word boundary matching
misspell_regex = {mis: re.compile(r'\b{}\b'.format(re.escape(mis)), flags=re.IGNORECASE)
                  for mis in corrections.keys()}

print(misspell_regex)

def count_misspells(text):
    """Count the total number of misspelled words in a text based on corrections dict."""
    return sum(len(regex.findall(text)) for regex in misspell_regex.values())

user_prompts['n_misspells'] = user_prompts['conversational'].apply(count_misspells)
user_prompts['misspell_rate'] = user_prompts['n_misspells'] / user_prompts['n_words']

# Display results
user_prompts[['user_id', 'gender', 'n_words', 'n_misspells', 'misspell_rate', 'conversational']]

{'orthogogal': re.compile('\\borthogogal\\b', re.IGNORECASE), 'Anothers': re.compile('\\bAnothers\\b', re.IGNORECASE), 'follwoing': re.compile('\\bfollwoing\\b', re.IGNORECASE), 'insted': re.compile('\\binsted\\b', re.IGNORECASE), 'addtion': re.compile('\\baddtion\\b', re.IGNORECASE), 'develope': re.compile('\\bdevelope\\b', re.IGNORECASE), 'familys': re.compile('\\bfamilys\\b', re.IGNORECASE), 'responsable': re.compile('\\bresponsable\\b', re.IGNORECASE), 'optimate': re.compile('\\boptimate\\b', re.IGNORECASE), 'esier': re.compile('\\besier\\b', re.IGNORECASE), 'sentencs': re.compile('\\bsentencs\\b', re.IGNORECASE), 'preveent': re.compile('\\bpreveent\\b', re.IGNORECASE), 'indexs': re.compile('\\bindexs\\b', re.IGNORECASE), 'hundrets': re.compile('\\bhundrets\\b', re.IGNORECASE), 'orthotogonal': re.compile('\\borthotogonal\\b', re.IGNORECASE), 'palatte': re.compile('\\bpalatte\\b', re.IGNORECASE), 'favoriting': re.compile('\\bfavoriting\\b', re.IGNORECASE), 'orignal': re.compile('\\b

,user_id,gender,n_words,n_misspells,misspell_rate,conversational
0,6,Man (cisgender),196,1,0.005102,"parsing data from python iterator, how it coul..."
1,8,Man (cisgender),36,0,0.000000,\n I am working on the problem of reconstru...
2,11,Woman (cisgender),193,0,0.000000,Can you adapt the following code so that inste...
3,15,Man (cisgender),36,0,0.000000,SET_ALL_TABLES action is currently not fetchin...
4,16,Woman (cisgender),542,7,0.012915,I want to use Dummy Hot encoding to replace th...
5,25,Man (cisgender),107,1,0.009346,whats the best way to encode and compress a ja...
6,28,Woman (cisgender),561,5,0.008913,I have a pandas dataframe like this:\n\nI want...
7,29,Woman (cisgender),61,0,0.000000,Now I want to bring judgement balancing into t...
9,31,Man (cisgender),92,0,0.000000,How can I make use of an ObservableHQDatabaseC...
10,34,Man (cisgender),1299,9,0.006928,Blender and Python. I have a collection of hun...


In [5]:
gender_means = user_prompts.groupby('gender')['misspell_rate'].mean()
gender_means

gender
Man (cisgender)      0.004976
Woman (cisgender)    0.014089
Name: misspell_rate, dtype: float64

In [6]:
from helpers.statistical_tests_new import compare_groups
from statsmodels.stats.multitest import multipletests

def groupwise_comparison_with_correction(df, columns, one_sided=False, direction="greater"):
    stat_vals = []
    dfs = []
    p_vals = []
    test_types = []
    directions = []
    one_sideds = []
    female_means = []
    male_means = []
    female_stds = []
    male_stds = []
    n_females = []
    n_males = []
    nonzero_females = []
    nonzero_males = []
    effsizes = []
    effsize_types = []
    cis = []
    for col in columns:
        result = compare_groups(df, col,group_column="gender", group_x="Man (cisgender)", group_y="Woman (cisgender)", one_sided=one_sided, direction=direction)
        stat_vals.append(result['test_statistic'])
        dfs.append(result['df'])
        p_vals.append(result['p_value'])
        test_types.append(result['test_type'])
        directions.append(result['direction'])
        one_sideds.append(result['one_sided'])
        female_means.append(result['mean_y'])
        male_means.append(result['mean_x'])
        female_stds.append(result['std_y'])
        male_stds.append(result['std_x'])
        n_females.append(result['n_y'])
        n_males.append(result['n_x'])
        nonzero_females.append(result['hits_y'])
        nonzero_males.append(result['hits_x']),
        effsizes.append(result['effect_size']),
        effsize_types.append(result['effect_size_type'])
        cis.append(result['effect_size_CI'])
    reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

    results = pd.DataFrame({
        'word': columns,
        'test type': test_types,
        'direction': directions,
        'one_sided': one_sideds,
        'df': dfs,
        'stat_value': stat_vals,
        'p_value': p_vals,
        'corrected p_value': pvals_corrected,
        'mean_f': female_means,
        'mean_m': male_means,
        'std_f': female_stds,
        'std_m': male_stds,
        'n_f': n_females,
        'n_m': n_males,
        'n_hits_f': nonzero_females,
        'n_hits_m': nonzero_males,
        'effsize': effsizes,
        'effsize_type': effsize_types,
        'ci': cis,
    })
    return results


In [7]:
from helpers.make_latex_table import create_latex_tables
results = groupwise_comparison_with_correction(user_prompts, ['misspell_rate'])
create_latex_tables(results, "latex/spelling_stats", "latex/spelling_eff")
results

,word,test type,direction,one_sided,df,stat_value,p_value,corrected p_value,mean_f,mean_m,std_f,std_m,n_f,n_m,n_hits_f,n_hits_m,effsize,effsize_type,ci
0,misspell_rate,Mann-Whitney U,greater,False,None,74.0,0.274431,0.274431,0.006726,0.00223,0.020586,0.005733,13,15,9,8,-0.241026,RBC,"[-0.6, 0.21]"
